# Metrics

> Fill in a module description here

In [ ]:
#| default_exp metrics

In [ ]:
#| export
import numpy as np
import warnings

In [ ]:
#| hide
from fastcore.test import test_close, test_eq

In [ ]:
#| export
def rmse(
    y_true: np.ndarray, # Ground truth target values.
    y_pred: np.ndarray, # Estimated target values.
    time_axis: int = 0 # Axis representing time or samples.
    ) -> np.ndarray: # Root Mean Squared Error for each channel.
    """
    Computes the Root Mean Square Error (RMSE) along a specified time axis.

    Calculates RMSE = sqrt(mean((y_pred - y_true)**2)) separately for each channel
    defined by the remaining axes.
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    if y_true.shape != y_pred.shape:
        raise ValueError(f"Input shapes must match. Got {y_true.shape} and {y_pred.shape}")

    # Ensure time_axis is valid
    if not (0 <= time_axis < y_true.ndim):
         raise ValueError(f"Invalid time_axis {time_axis} for array with {y_true.ndim} dimensions")

    # Calculate RMSE
    try:
        rmse_val = np.sqrt(np.mean((y_pred - y_true)**2, axis=time_axis))
    except FloatingPointError as e:
         warnings.warn(f"Floating point error during RMSE calculation: {e}. Check for NaNs or Infs.", RuntimeWarning)
         # Decide how to handle - perhaps return NaN or raise a more specific error
         # For now, re-raising but ideally might return NaN or a specific value
         raise e
    return rmse_val

In [ ]:
# Example usage for rmse

# 1D case
y_t1 = np.array([1, 2, 3, 4, 5])
y_p1 = np.array([1.1, 2.1, 3.1, 4.1, 5.1]) # Small error
rmse_val_1d = rmse(y_t1, y_p1)
print(f"RMSE (1D): {rmse_val_1d}") # Should be ~0.1

# 2D case (e.g., time x features)
y_t2 = np.array([[1, 10], [2, 20], [3, 30]])
y_p2 = np.array([[1, 11], [2, 21], [3, 31]]) # Error=0 in col 0, Error=1 in col 1
rmse_val_2d = rmse(y_t2, y_p2, time_axis=0)
print(f"RMSE (2D, time_axis=0): {rmse_val_2d}") # Should be [0., 1.]

# 3D case (e.g., batch x time x features)
y_t3 = np.random.rand(2, 10, 3) # batch=2, time=10, features=3
y_p3 = y_t3 + np.random.randn(2, 10, 3) * 0.1
rmse_val_3d = rmse(y_t3, y_p3, time_axis=1) # Calculate RMSE over time axis
print(f"RMSE (3D, time_axis=1, shape={rmse_val_3d.shape}): {rmse_val_3d}") # Should have shape (2, 3)

RMSE (1D): 0.09999999999999991
RMSE (2D, time_axis=0): [0. 1.]
RMSE (3D, time_axis=1, shape=(2, 3)): [[0.09308798 0.11013668 0.11058777]
 [0.08573954 0.06386016 0.06679621]]


In [ ]:
#| export
def nrmse(
    y_true: np.ndarray, # Ground truth target values.
    y_pred: np.ndarray, # Estimated target values.
    time_axis: int = 0, # Axis representing time or samples.
    std_tolerance: float = 1e-9 # Minimum standard deviation allowed for y_true to avoid division by zero.
    ) -> np.ndarray: # Normalized Root Mean Squared Error for each channel.
    """
    Computes the Normalized Root Mean Square Error (NRMSE).

    Calculates NRMSE = RMSE / std(y_true) separately for each channel.
    Returns NaN for channels where std(y_true) is close to zero (below std_tolerance).
    """
    rmse_val = rmse(y_true, y_pred, time_axis=time_axis)
    std_true = np.std(y_true, axis=time_axis)

    # Initialize nrmse_val with NaNs or another placeholder
    nrmse_val = np.full_like(std_true, fill_value=np.nan, dtype=np.float64)

    # Identify channels with standard deviation above the tolerance
    valid_std_mask = std_true > std_tolerance

    # Calculate NRMSE only for valid channels
    # Using np.divide with 'where' handles division by zero gracefully if needed,
    # but the explicit check is clearer here.
    if np.any(valid_std_mask):
        nrmse_val[valid_std_mask] = rmse_val[valid_std_mask] / std_true[valid_std_mask]

    # Warn if any channels had std below tolerance
    if not np.all(valid_std_mask):
        warnings.warn(f"Standard deviation of y_true is below tolerance ({std_tolerance}) for some channels. NRMSE set to NaN for these channels.", RuntimeWarning)

    return nrmse_val

In [ ]:
# Example usage for nrmse

# 1D case
y_t1 = np.array([1, 2, 3, 4, 5])
y_p1 = y_t1 + 0.1 # Constant offset error
nrmse_val_1d = nrmse(y_t1, y_p1)
print(f"NRMSE (1D): {nrmse_val_1d}") # RMSE is 0.1, std(y_t1) is sqrt(2). NRMSE = 0.1 / sqrt(2)

# 2D case
y_t2 = np.array([[1, 10], [2, 20], [3, 30]])
y_p2 = np.array([[1, 11], [2, 21], [3, 31]]) # Error=0 in col 0, Error=1 in col 1
# Std dev col 0: ~0.816, Std dev col 1: ~8.165
# RMSE col 0: 0, RMSE col 1: 1
# NRMSE col 0: 0 / 0.816 = 0
# NRMSE col 1: 1 / 8.165 = ~0.122
nrmse_val_2d = nrmse(y_t2, y_p2, time_axis=0)
print(f"NRMSE (2D, time_axis=0): {nrmse_val_2d}") # Should be approx [0., 0.122]

# Case with zero standard deviation
y_t_const = np.array([5, 5, 5, 5])
y_p_const = np.array([5, 5, 5, 6]) # RMSE = 0.5
nrmse_val_const = nrmse(y_t_const, y_p_const)
print(f"NRMSE (Constant y_true): {nrmse_val_const}") # Should be NaN with a warning

NRMSE (1D): 0.07071067811865468
NRMSE (2D, time_axis=0): [0.         0.12247449]
NRMSE (Constant y_true): nan


/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_17861/2787241071.py:31: RuntimeWarning: Standard deviation of y_true is below tolerance (1e-09) for some channels. NRMSE set to NaN for these channels.
  warnings.warn(f"Standard deviation of y_true is below tolerance ({std_tolerance}) for some channels. NRMSE set to NaN for these channels.", RuntimeWarning)


In [ ]:
#| export
def fit_index(
    y_true: np.ndarray, # Ground truth target values.
    y_pred: np.ndarray, # Estimated target values.
    time_axis: int = 0, # Axis representing time or samples.
    std_tolerance: float = 1e-9 # Minimum standard deviation allowed for y_true.
    ) -> np.ndarray: # Fit index (in percent) for each channel.
    """
    Computes the Fit Index (FIT) commonly used in System Identification.

    Calculates FIT = 100 * (1 - NRMSE) separately for each channel.
    Returns NaN for channels where NRMSE could not be calculated (e.g., std(y_true) near zero).
    """
    nrmse_val = nrmse(y_true, y_pred, time_axis=time_axis, std_tolerance=std_tolerance)

    # Fit index calculation, handles potential NaNs from nrmse
    fit_val = 100.0 * (1.0 - nrmse_val)

    return fit_val

In [ ]:
# Example usage for fit_index

# 1D case (using previous example)
# NRMSE = 0.1 / sqrt(2) approx 0.0707
# FIT = 100 * (1 - 0.0707) approx 92.93
fit_val_1d = fit_index(y_t1, y_p1)
print(f"Fit Index (1D): {fit_val_1d}")

# 2D case (using previous example)
# NRMSE approx [0., 0.122]
# FIT approx [100 * (1 - 0), 100 * (1 - 0.122)] = [100, 87.8]
fit_val_2d = fit_index(y_t2, y_p2, time_axis=0)
print(f"Fit Index (2D, time_axis=0): {fit_val_2d}")

# Constant case (using previous example)
# NRMSE is NaN
# FIT should also be NaN
fit_val_const = fit_index(y_t_const, y_p_const)
print(f"Fit Index (Constant y_true): {fit_val_const}")

Fit Index (1D): 92.92893218813452
Fit Index (2D, time_axis=0): [100.          87.75255129]
Fit Index (Constant y_true): nan


/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_17861/2787241071.py:31: RuntimeWarning: Standard deviation of y_true is below tolerance (1e-09) for some channels. NRMSE set to NaN for these channels.
  warnings.warn(f"Standard deviation of y_true is below tolerance ({std_tolerance}) for some channels. NRMSE set to NaN for these channels.", RuntimeWarning)


In [ ]:
#| export
def mae(
    y_true: np.ndarray, # Ground truth target values.
    y_pred: np.ndarray, # Estimated target values.
    time_axis: int = 0 # Axis representing time or samples.
    ) -> np.ndarray: # Mean Absolute Error for each channel.
    """
    Computes the Mean Absolute Error (MAE) along a specified time axis.

    Calculates MAE = mean(abs(y_pred - y_true)) separately for each channel
    defined by the remaining axes.
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    if y_true.shape != y_pred.shape:
        raise ValueError(f"Input shapes must match. Got {y_true.shape} and {y_pred.shape}")

    # Ensure time_axis is valid
    if not (0 <= time_axis < y_true.ndim):
         raise ValueError(f"Invalid time_axis {time_axis} for array with {y_true.ndim} dimensions")

    # Calculate MAE
    try:
        mae_val = np.mean(np.abs(y_pred - y_true), axis=time_axis)
    except FloatingPointError as e:
         warnings.warn(f"Floating point error during MAE calculation: {e}. Check for NaNs or Infs.", RuntimeWarning)
         # Decide how to handle - perhaps return NaN or raise a more specific error
         raise e
    return mae_val

In [ ]:
# Example usage for mae

# 1D case
y_t1 = np.array([1, 2, 3, 4, 5])
y_p1 = np.array([1.1, 2.1, 3.1, 4.1, 5.1]) # Constant error of 0.1
mae_val_1d = mae(y_t1, y_p1)
print(f"MAE (1D): {mae_val_1d}") # Should be 0.1

y_p2 = np.array([0, 1, 2, 3, 4]) # Constant error of -1 -> abs error 1
mae_val_1d_neg = mae(y_t1, y_p2)
print(f"MAE (1D, negative err): {mae_val_1d_neg}") # Should be 1.0

# 2D case (e.g., time x features)
y_t2 = np.array([[1, 10], [2, 20], [3, 30]])
y_p3 = np.array([[1, 11], [2, 21], [3, 31]]) # Error=0 in col 0, Error=1 in col 1
mae_val_2d = mae(y_t2, y_p3, time_axis=0)
print(f"MAE (2D, time_axis=0): {mae_val_2d}") # Should be [0., 1.]

MAE (1D): 0.09999999999999991
MAE (1D, negative err): 1.0
MAE (2D, time_axis=0): [0. 1.]


In [ ]:
#| export
def r_squared(
    y_true: np.ndarray, # Ground truth target values.
    y_pred: np.ndarray, # Estimated target values.
    time_axis: int = 0, # Axis representing time or samples.
    std_tolerance: float = 1e-9 # Minimum standard deviation allowed for y_true.
    ) -> np.ndarray: # R-squared (coefficient of determination) for each channel.
    """
    Computes the R-squared (coefficient of determination) score.

    Calculates R^2 = 1 - NRMSE^2 separately for each channel.
    Returns NaN for channels where NRMSE could not be calculated (e.g., std(y_true) near zero).
    A constant model that always predicts the mean of y_true would get R^2=0.
    """
    nrmse_val = nrmse(y_true, y_pred, time_axis=time_axis, std_tolerance=std_tolerance)

    # R^2 calculation, handles potential NaNs from nrmse
    # Ensure we don't take the square of NaN resulting in NaN where 0 might be expected if rmse is 0
    r2 = 1.0 - nrmse_val**2

    return r2

In [ ]:
# Example usage for r_squared

# 1D case (using previous nrmse example)
# NRMSE = 0.1 / sqrt(2) approx 0.0707
# R^2 = 1 - (0.0707)^2 approx 1 - 0.005 = 0.995
y_t1 = np.array([1, 2, 3, 4, 5])
y_p1 = np.array([1.1, 2.1, 3.1, 4.1, 5.1])
r2_val_1d = r_squared(y_t1, y_p1)
print(f"R-squared (1D): {r2_val_1d}")

# Perfect prediction
r2_perfect = r_squared(y_t1, y_t1)
print(f"R-squared (1D, perfect): {r2_perfect}") # Should be 1.0

# 2D case (using previous nrmse example)
# NRMSE approx [0., 0.122]
# R^2 approx [1 - 0^2, 1 - 0.122^2] = [1, 1 - 0.0149] = [1, 0.9851]
y_t2 = np.array([[1, 10], [2, 20], [3, 30]])
y_p2 = np.array([[1, 11], [2, 21], [3, 31]])
r2_val_2d = r_squared(y_t2, y_p2, time_axis=0)
print(f"R-squared (2D, time_axis=0): {r2_val_2d}")

# Constant case (using previous nrmse example)
# NRMSE is NaN
# R^2 should also be NaN
y_t_const = np.array([5, 5, 5, 5])
y_p_const = np.array([5, 5, 5, 6])
r2_val_const = r_squared(y_t_const, y_p_const)
print(f"R-squared (Constant y_true): {r2_val_const}") # Should be NaN with a warning from nrmse

R-squared (1D): 0.995
R-squared (1D, perfect): 1.0
R-squared (2D, time_axis=0): [1.    0.985]
R-squared (Constant y_true): nan


/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_17861/2787241071.py:31: RuntimeWarning: Standard deviation of y_true is below tolerance (1e-09) for some channels. NRMSE set to NaN for these channels.
  warnings.warn(f"Standard deviation of y_true is below tolerance ({std_tolerance}) for some channels. NRMSE set to NaN for these channels.", RuntimeWarning)


In [ ]:
#| hide
# --- Tests ---

# Test data
y_t_1d = np.array([1., 2., 3., 4., 5.])
y_p_1d_perfect = np.array([1., 2., 3., 4., 5.])
y_p_1d_offset = np.array([1.1, 2.1, 3.1, 4.1, 5.1]) # Error=0.1
y_p_1d_noise = np.array([1.1, 1.9, 3.1, 4.1, 4.9]) # Errors: 0.1, -0.1, 0.1, 0.1, -0.1
y_p_1d_neg_offset = np.array([0., 1., 2., 3., 4.]) # Error=-1.0

y_t_2d = np.array([[1., 10.], [2., 20.], [3., 30.]]) # time x features
y_p_2d_perfect = np.array([[1., 10.], [2., 20.], [3., 30.]])
y_p_2d_offset = np.array([[1., 11.], [2., 21.], [3., 31.]]) # Error=[0, 1]
y_p_2d_mixed_err = np.array([[1.1, 10.], [1.9, 21.], [3.1, 30.]]) # Errors: [0.1, 0], [-0.1, 1], [0.1, 0]

y_t_const = np.array([5., 5., 5.])
y_p_const_err = np.array([5., 5., 6.]) # Error=[0, 0, 1]

# RMSE Tests
test_close(rmse(y_t_1d, y_p_1d_perfect), 0.0)
test_close(rmse(y_t_1d, y_p_1d_offset), 0.1)
test_close(rmse(y_t_2d, y_p_2d_perfect, time_axis=0), np.array([0., 0.]))
test_close(rmse(y_t_2d, y_p_2d_offset, time_axis=0), np.array([0., 1.]))

# NRMSE Tests
std_1d = np.std(y_t_1d)
nrmse_1d_offset_expected = 0.1 / std_1d
test_close(nrmse(y_t_1d, y_p_1d_offset), nrmse_1d_offset_expected)

std_2d = np.std(y_t_2d, axis=0)
nrmse_2d_offset_expected = np.array([0., 1.]) / std_2d
test_close(nrmse(y_t_2d, y_p_2d_offset, time_axis=0), nrmse_2d_offset_expected)

# Test NRMSE with zero std - should return NaN and warn
nrmse_const = nrmse(y_t_const, y_p_const_err)
test_eq(np.isnan(nrmse_const).all(), True) # Check if all elements are NaN if multi-output

# Fit Index Tests
fit_1d_offset_expected = 100.0 * (1.0 - nrmse_1d_offset_expected)
test_close(fit_index(y_t_1d, y_p_1d_offset), fit_1d_offset_expected)

fit_2d_offset_expected = 100.0 * (1.0 - nrmse_2d_offset_expected)
test_close(fit_index(y_t_2d, y_p_2d_offset, time_axis=0), fit_2d_offset_expected)

# Test Fit Index with zero std - should return NaN
fit_const = fit_index(y_t_const, y_p_const_err)
test_eq(np.isnan(fit_const).all(), True) # Check if all elements are NaN

# MAE Tests
test_close(mae(y_t_1d, y_p_1d_perfect), 0.0)
test_close(mae(y_t_1d, y_p_1d_offset), 0.1)
test_close(mae(y_t_1d, y_p_1d_neg_offset), 1.0)
test_close(mae(y_t_1d, y_p_1d_noise), np.mean([0.1, 0.1, 0.1, 0.1, 0.1]))
test_close(mae(y_t_2d, y_p_2d_perfect, time_axis=0), np.array([0., 0.]))
test_close(mae(y_t_2d, y_p_2d_offset, time_axis=0), np.array([0., 1.]))
# MAE for mixed errors: abs errors are [[0.1, 0], [0.1, 1], [0.1, 0]]
# Mean along axis 0: [mean(0.1, 0.1, 0.1), mean(0, 1, 0)] = [0.1, 1/3]
test_close(mae(y_t_2d, y_p_2d_mixed_err, time_axis=0), np.array([0.1, 1./3.]))

# R-squared Tests
test_close(r_squared(y_t_1d, y_p_1d_perfect), 1.0)
r2_1d_offset_expected = 1.0 - nrmse_1d_offset_expected**2
test_close(r_squared(y_t_1d, y_p_1d_offset), r2_1d_offset_expected)

r2_2d_offset_expected = 1.0 - nrmse_2d_offset_expected**2
test_close(r_squared(y_t_2d, y_p_2d_offset, time_axis=0), r2_2d_offset_expected)

# Test R-squared with zero std - should return NaN
r2_const = r_squared(y_t_const, y_p_const_err)
test_eq(np.isnan(r2_const).all(), True) # Check if all elements are NaN

/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_17861/2787241071.py:31: RuntimeWarning: Standard deviation of y_true is below tolerance (1e-09) for some channels. NRMSE set to NaN for these channels.
  warnings.warn(f"Standard deviation of y_true is below tolerance ({std_tolerance}) for some channels. NRMSE set to NaN for these channels.", RuntimeWarning)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()